In [1]:
#importing the dataset
path = "/kaggle/input/finaldataset/finalMentalHealthDataset-question-response.csv"

import pandas as pd
df = pd.read_csv(path)
df.head()

,Unnamed: 0,question,response
0,0,What is a panic attack?\n,Panic attacks come on suddenly and involve int...
1,1,What are symptoms of panic attack vs. anxiety ...,Panic attacks and anxiety attacks can share so...
2,2,What are the types of Mental Illness?\n,There are many different conditions that are r...
3,3,What does mental-illness mean?\n,Mental illness is a group of biological brain ...
4,4,How can you treat mental illness?\n,The treatment of mental illness is a multi-fac...


In [2]:
df = df.dropna(subset=['question', 'response'])

In [3]:
df.shape

(19888, 3)

In [4]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

In [5]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df[['question', 'response']])
val_dataset = Dataset.from_pandas(val_df[['question', 'response']])


In [6]:
from transformers import BartTokenizer, BartModel

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
def tokenize_function(examples):
    inputs = examples['question']
    targets = examples['response']
    model_inputs = tokenizer(inputs, max_length=512, padding = 'max_length', truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, padding = 'max_length', truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/15910 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3978 [00:00<?, ? examples/s]

In [7]:
import wandb

# Set the API key from environment variable or directly in the code
wandb.login(key="d4f16d74c6ca76529bb0c4ac6fb263004c167869")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
import os
from transformers import BartForConditionalGeneration, TrainingArguments, Trainer
from pathlib import Path  # Import Path

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Check for any checkpoint
last_checkpoint = None
if os.path.isdir('/kaggle/input/30_epoch_bart/transformers/default/1/results'):
    checkpoints = [str(x) for x in Path('/kaggle/input/30_epoch_bart/transformers/default/1/results').glob('checkpoint-*')]
    if checkpoints:
        last_checkpoint = max(checkpoints, key=os.path.getctime)  # Get the latest checkpoint


training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=50,
    evaluation_strategy="epoch",
    save_total_limit=3,
    save_strategy="epoch",  
    logging_dir='./logs',
    fp16=True,
    dataloader_num_workers=4,
    gradient_accumulation_steps=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

if last_checkpoint:
    print(f"Resuming from checkpoint {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("The training is starting now:")
    trainer.train()



model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Resuming from checkpoint /kaggle/input/30_epoch_bart/transformers/default/1/results/checkpoint-12425


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
/opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded

Epoch,Training Loss,Validation Loss
25,0.185300,0.162221
27,0.175800,0.155391
29,0.160900,0.147300
31,0.148600,0.144113
33,0.136600,0.140974
35,0.128200,0.138529
37,0.122200,0.135769
39,0.116700,0.134570
41,0.111600,0.134401
43,0.108500,0.134478


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: 

In [9]:
# Save model and tokenizer
model.save_pretrained('/kaggle/working/final_model')
tokenizer.save_pretrained('/kaggle/working/final_tokenizer')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/kaggle/working/final_tokenizer/tokenizer_config.json',
 '/kaggle/working/final_tokenizer/special_tokens_map.json',
 '/kaggle/working/final_tokenizer/vocab.json',
 '/kaggle/working/final_tokenizer/merges.txt',
 '/kaggle/working/final_tokenizer/added_tokens.json')

In [10]:
eval_results = trainer.evaluate(eval_dataset=val_dataset)  # Replace with your dataset

print(f"Evaluation results: {eval_results}")

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Evaluation results: {'eval_loss': 0.13346421718597412, 'eval_runtime': 128.9144, 'eval_samples_per_second': 30.858, 'eval_steps_per_second': 1.932, 'epoch': 49.97487437185929}


In [11]:
!pip install rouge-score


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... - done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9d16226ccb9e2941f56a044b8dafb5955c8e2c72682ef07bbcbc712bde7424c1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [12]:
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import torch

# Load the model and tokenizer
model = BartForConditionalGeneration.from_pretrained('/kaggle/working/final_model')
tokenizer = BartTokenizer.from_pretrained('/kaggle/working/final_tokenizer')

def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

def calculate_bleu_score(reference_texts, generated_text):
    references = [ref.split() for ref in reference_texts]
    candidate = generated_text.split()
    score = sentence_bleu(references, candidate)
    return score

def calculate_rouge_score(reference_text, generated_text):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_text, generated_text)
    return scores

def calculate_f1_score(rouge_scores):
    f1_scores = {}
    for key, score in rouge_scores.items():
        precision = score.precision
        recall = score.recall
        if precision + recall > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
        else:
            f1 = 0.0
        f1_scores[key + '_f1'] = f1
    return f1_scores

def evaluate_model(texts, model, tokenizer):
    results = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=512,  
                num_beams=5,    
                early_stopping=True,
                pad_token_id=tokenizer.eos_token_id)
            
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Calculate metrics
        perplexity = calculate_perplexity(model, tokenizer, text)
        bleu_score = calculate_bleu_score([text], generated_text)
        rouge_scores = calculate_rouge_score(text, generated_text)
        f1_scores = calculate_f1_score(rouge_scores)
        
        results.append({
            'generated_text': generated_text,
            'perplexity': perplexity,
            'bleu_score': bleu_score,
            'rouge_scores': rouge_scores,
            'f1_scores': f1_scores
        })
    return results

# Example texts to evaluate
texts = ["I'm feeling really stressed out. Do you have any advice for managing stress?",
    "Can you help me understand how to deal with anxiety?",
    "I’m struggling with my work-life balance. What should I do?"]

evaluation_results = evaluate_model(texts, model, tokenizer)
for result in evaluation_results:
    print(f"Generated Text: {result['generated_text']}")
    print(f"Perplexity: {result['perplexity']}")
    print(f"BLEU Score: {result['bleu_score']}")
    print(f"ROUGE Scores: {result['rouge_scores']}")
    print(f"F1 Scores: {result['f1_scores']}")


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Generated Text: Stress is a normal part of life, but it's important to find healthy ways to manage it. Let's work together to identify the sources of your stress and develop coping strategies that work for you.
Perplexity: 7.567426681518555
BLEU Score: 0.41113361690051975
ROUGE Scores: {'rouge1': Score(precision=0.13513513513513514, recall=0.35714285714285715, fmeasure=0.19607843137254902), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.08108108108108109, recall=0.21428571428571427, fmeasure=0.11764705882352942)}
F1 Scores: {'rouge1_f1': 0.19607843137254902, 'rouge2_f1': 0.0, 'rougeL_f1': 0.11764705882352942}
Generated Text: This question is a great one! Depression is a treatable condition, and there are many evidence-based treatments available that can help you manage your symptoms. Here are some of the treatment options:

1. Talk to a therapist: Talk to someone you trust about your anxiety. Sharing your feelings with someone who specializes in a